In [ ]:
import numpy as np              # NumPy, for working with arrays/tensors 
import time                     # For measuring time
import random                   # Python's random library
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# PyTorch libraries:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.preprocessing import normalize

if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available :(")
  device = torch.device("cpu")


: 

In [ ]:
LABELS_FILEPATH = "./SMAP MSL/labeled_anomalies.csv"
TRAINSET_FILEPATH = "./SMAP MSL/data/data/train"
TESTSET_FILEPATH = "./SMAP MSL/data/data/test"

: 

In [ ]:
labels = pd.read_csv(LABELS_FILEPATH)

: 

In [ ]:
labels.info()

: 

In [ ]:
labels.head(50)

: 

In [ ]:
#training params
BATCH_SIZE = 128
INPUT_LENGTH = 100
INTERMEDIATE_LENGTH = 24
ALPHA  =0.1
DIMENSIONS  = 25 
lr = 0.005
#optimizer = torch.optim.Adam(lr = lr )
loss = nn.MSELoss()

: 

In [ ]:
#data loading 
import os
train_datas = {}
test_datas = {}
arranged_train_datas = {}
arranged_test_datas = {}
# Iterate directory
for path in os.listdir(TRAINSET_FILEPATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(TRAINSET_FILEPATH, path)):
        train_data = np.load(os.path.join(TRAINSET_FILEPATH, path))
        train_datas[path] = train_data
for path in os.listdir(TESTSET_FILEPATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(TESTSET_FILEPATH, path)):
        test_data = np.load(os.path.join(TESTSET_FILEPATH, path))
        test_datas[path] = test_data

for train_data in train_datas:
    print(train_datas[train_data ].shape)



: 

In [ ]:
#4.1 data normalization 
#1 - training normalization 
for train_data in train_datas:
    train_datas[train_data] = normalize(train_datas[train_data])
    train_datas[train_data] =  train_datas[train_data] * 2 - 1
#2 - test normalization
test_means = {}
test_variances = {}

for test_data in test_datas:
    mean = 0
    variance = 1
    # generate expected value of for discrete datas
    expected_values = {}
    for row in test_datas[test_data]:
        for element in row:
            if element in expected_values:
                expected_values[element] += 1
            else:
                expected_values[element] = 0
                
    sum = 0
    for keys in expected_values:
        sum += expected_values[keys] 

    for keys in expected_values:
        expected_values[keys] /= sum

    #now we got expected values
    #iterate through data
    for row in test_datas[test_data]:
        for element in row:
            mean = (1 - ALPHA) * mean + ALPHA * expected_values[element]
            e_x_2 = 0
            if (element*element) in expected_values:
                e_x_2 = expected_values[element*element]
            variance = (1 - ALPHA) * variance + ALPHA * ( e_x_2 - expected_values[element] * expected_values[element]  )
            test_means[test_data + "_" + str(row) + "_" + str(element)] = mean 
            test_variances[test_data + "_" + str(row) + "_" + str(element)] = variance 
    



                 
            



: 

In [ ]:
test_datas

: 

In [ ]:
#4.2 temporal correlation
#inout sub block 


class InputSubBlock(nn.Module):

    def __init__(self):
        super(InputSubBlock, self).__init__()
        self.fc1 = nn.Linear(INPUT_LENGTH, 50)  # 5*5 from image dimension
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50, 50)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
#cascade sub block 
class CascadeSubBlock(nn.Module):

    def __init__(self):
        super(CascadeSubBlock, self).__init__()
        self.fc1 = nn.Linear(INPUT_LENGTH, 50)  # 5*5 from image dimension
        self.fc2 = nn.Linear(50, INPUT_LENGTH)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
#forecasting sub block
class ForecastingSubBlock(nn.Module):

    def __init__(self):
        super(ForecastingSubBlock, self).__init__()
        self.fc1 = nn.Linear(INPUT_LENGTH, 50)  # 5*5 from image dimension
        self.fc2 = nn.Linear(50, INTERMEDIATE_LENGTH)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
#MLPBlock
class MLPBlock(nn.Module):

    def __init__(self):
        super(MLPBlock, self).__init__()
        self.input = InputSubBlock()
        self.cascade = CascadeSubBlock()
        self.forecasting = ForecastingSubBlock()

    def forward(self, x):
        x = self.input(x)
        cascade = self.cascade(x)
        forecast = self.forecasting(x)
        return cascade , forecast
class TemporalModule(nn.Module):
    def __init__(self):
        super(TemporalModule, self).__init__()
        self.mlp1 = MLPBlock() 
        self.mlp2 = MLPBlock() 

    def forward(self, input):
        mlp1_out , forecast = self.mlp1(input)
        new_input = mlp1_out - input 
        mlp2_out , forecast_2 = self.mlp2(new_input)
        return (forecast + forecast_2)

s = TemporalModule()

: 